In [1]:
import pandas as pd
from tqdm import tqdm

from concurrent.futures import ThreadPoolExecutor

In [2]:
# Where the JSON files are located
source = '../data/source/newsclust.csv'
df_file_name = '../data/source/df-entities-sentiment.pickle.gz'

In [3]:
df = pd.read_csv("../data/source/newsclust.csv")
df = df.query("site != 'cbn.com'")
print(len(df))
df = df.reset_index([0], drop="true")
# Uncomment to run on a very small sample
# df = df.head(2)

103262


In [4]:
from bias import SimplifiedBias

df['bias'] = df.apply(lambda row: SimplifiedBias.get_simplified_bias_for_domain(row['site']).value, axis=1)
df.head(2)

,Unnamed: 0,date,site,text,title,url,bias
0,0,2015-01-29T23:14:00.000+02:00,washingtonexaminer.com,Class action filed over United’s ‘low fare gua...,Class action filed over United’s ‘low fare gua...,http://www.washingtonexaminer.com/class-action...,1
1,1,2015-01-23T02:00:00.000+02:00,nydailynews.com,Jupiterimages/Getty Images/Goodshoot RF Snuggl...,Portland pro cuddler hosts ‘Cuddle Con’ on Val...,http://www.nydailynews.com/news/national/portl...,1


In [ ]:
from google.cloud.gapic.language.v1beta2 import enums
from google.cloud.gapic.language.v1beta2 import language_service_client
from google.cloud.proto.language.v1beta2 import language_service_pb2
import six
import sys

# taken from https://cloud.google.com/natural-language/docs/beta#analyze-entity-sentiment

def entity_sentiment_text(text):
    """Detects entity sentiment in the provided text."""
    language_client = language_service_client.LanguageServiceClient()
    document = language_service_pb2.Document()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document.content = text.encode('utf-8')
    document.type = enums.Document.Type.PLAIN_TEXT

    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = language_client.analyze_entity_sentiment(
        document, encoding)

    # for entity in result.entities:
    #     print('Mentions: ')
    #    print(u'Name: "{}"'.format(entity.name))
    #     for mention in entity.mentions:
    #         print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
    #         print(u'  Content : {}'.format(mention.text.content))
    #         print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
    #         print(u'  Sentiment : {}'.format(mention.sentiment.score))
    #         print(u'  Type : {}'.format(mention.type))
    #     print(u'Salience: {}'.format(entity.salience))
    #     print(u'Sentiment: {}\n'.format(entity.sentiment))
        
    return result

# from time import sleep
def safe_entity_sentiment_text(pbar, text):
    try:
        pbar.update(1)
        return entity_sentiment_text(text)
        # sleep(0.500)
        # return 1
    except:
        return None

def parallel_get_entities(texts):
    with tqdm(total=len(texts)) as pbar:
        with ThreadPoolExecutor(max_workers=50) as pool:
            return pool.map(lambda text: safe_entity_sentiment_text(pbar, text), texts)

In [ ]:
# entities = [safe_entity_sentiment_text(text) for text in tqdm(df['text'])]
entities = list(parallel_get_entities(df['text']))

100%|█████████▉| 103217/103262 [01:54<00:00, 1231.40it/s]

In [ ]:
df['entities'] = entities
len(entities)

In [ ]:
df.head()

In [ ]:
df.to_pickle(df_file_name, compression="gzip")
df2 = pd.read_pickle(df_file_name)

In [ ]:
df2['entities'][0].entities[0].name